In [30]:
import psycopg2
import pandas as pd 
import numpy as np

# ERP

Locaction

In [31]:
# Vì phần dữ liệu ERP có nhiều bảng nên sẽ ETL từng bảng, xử lý các bảng Dim trước
# Location Extraction
# Kết nối đến CSDL ERP
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    #Tạo con trỏ để thực thi câu lệnh
    cur = conn.cursor()
    #Thực thi câu lệnh lấy dữ liệu từ bảng Location
    cur.execute('''SELECT *
    FROM public."Location";''')
    #Lưu kết quả truy vấn vào biến res
    res = cur.fetchall()
    #Chuyển thành dataframe Location
    location=pd.DataFrame(res,columns = ['location_id','location_name'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
location

,location_id,location_name
0,1,New York
1,2,Atlanta
2,3,Chicago
3,4,Dallas
4,5,Denver
5,6,Los Angeles
6,7,Seattle
7,8,Toronto
8,9,Montreal
9,10,Vancouver


In [32]:
# Location Transform
# Hiển thị Location info 
location.info()
#Ta thấy dữ liệu trong location bao gồm các cột, các dòng dữ liệu đã đúng định dạng, làm sạch theo như thiết kế W_Location_D
# Kết thúc Location Transform

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   location_id    12 non-null     int64 
 1   location_name  12 non-null     object
dtypes: int64(1), object(1)
memory usage: 324.0+ bytes


In [33]:
# Location Load
#Kết nối đến erp dw _ nơi lưu trữ dữ liệu của Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện vòng for để insert giá trị của dataframe location vào trong bảng Dim Location x[0] ~ location_id, x[1] ~ location_name
    for x in location.values:
        cur.execute('''INSERT INTO public.w_location_d(location_id, location_name)
        VALUES (%s, %s);''',(x[0],x[1]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Sales Agent

In [34]:
# Sales Agent Extract
#Kết nối vào CSDL ERP
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực thi câu lệnh lấy tất cả giá trị trong bảng SalesAgent
    cur.execute('''SELECT *
	FROM public."SalesAgent";''')
    res = cur.fetchall()
    sales_agent=pd.DataFrame(res,columns = ['sales_agent_id','sales_agent_name','sales_agent_state','country','record_active'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
sales_agent

,sales_agent_id,sales_agent_name,sales_agent_state,country,record_active
0,1338,Kassandra Mccullough,MA,USA,None
1,1339,Malik Hampton,TX,USA,None
2,1340,Trent Ross,NC,USA,None
3,1341,Makhi Williamson,MD,USA,None
4,1342,Lina Davies,FL,USA,None
...,...,...,...,...,...
495,1616,Jaquan Howe,GA,USA,None
496,1617,Giada Davila,IL,USA,None
497,1618,Jensen Huff,MB,Canada,None
498,1619,Branden Griffin,MS,USA,None


In [35]:
#SalesAgent Transform
#Lọc lấy các cột cần thiết
sales_agent.drop(columns = 'record_active',inplace = True)
#Kiểm tra giá trị null
sales_agent.info()
sales_agent
#Ta thấy không có giá trị null nào và đang đúng định dạng mong muốn
#Kết thúc SalesAgent Transform

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sales_agent_id     500 non-null    int64 
 1   sales_agent_name   500 non-null    object
 2   sales_agent_state  500 non-null    object
 3   country            500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB


,sales_agent_id,sales_agent_name,sales_agent_state,country
0,1338,Kassandra Mccullough,MA,USA
1,1339,Malik Hampton,TX,USA
2,1340,Trent Ross,NC,USA
3,1341,Makhi Williamson,MD,USA
4,1342,Lina Davies,FL,USA
...,...,...,...,...
495,1616,Jaquan Howe,GA,USA
496,1617,Giada Davila,IL,USA
497,1618,Jensen Huff,MB,Canada
498,1619,Branden Griffin,MS,USA


In [36]:
#Sales Agent Load
#Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_sales_agent 
    for x in sales_agent.values:
        cur.execute('''INSERT INTO public.w_sales_agent_d(sales_agent_id,
        sales_agent_name, sales_agent_state, country)
        VALUES (%s, %s, %s, %s);;''',(x[0],x[1],x[2],x[3]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Sales Class

In [37]:
#Extract Sales Class
#Kết nối vào CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện truy vấn lấy tất cả giá trị trong bảng SalesClass
    cur.execute('''SELECT *
	FROM public."SalesClass";''')
    #Lưu kết quả thực thi vào biến res
    res = cur.fetchall()
    #Chuyển thành dataframe sales_class
    sales_class=pd.DataFrame(res,columns = ['sales_class_id','sales_class_desc','base_price'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
sales_class

,sales_class_id,sales_class_desc,base_price
0,1,Debit Smart,1.6000
1,2,Credit Smart,1.6000
2,3,Debit NoSmart,0.8000
3,4,Credit NoSmart,0.8000
4,5,Prepaid NoSmart,0.8000
5,6,Loyalty NoSmart,0.8000


In [38]:
# sales_class Transform
#Ta thấy dữ liệu có 6 dòng, đúng định dạng, đúng thiết kế các cột cần thiết
# Kết thúc sales_class Transform

In [39]:
#Salse Class Load
#Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_sales_class_d
    for x in sales_class.values:
        cur.execute('''INSERT INTO public.w_sales_class_d(
	sales_class_id, sales_class_desc,base_price)
	VALUES (%s, %s,%s);''',(x[0],x[1],x[2]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Customer

In [40]:
# customer extract
#Kết nối vào CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện truy vấn lấy tất cả giá trị trong bảng Customer
    cur.execute('''SELECT *
	FROM public."Customer";''')
    res = cur.fetchall()
    #Chuyển thành dataframe
    customer=pd.DataFrame(res,columns = ['cust_key','cust_name','active','address_1','address_2','address_3','city','country',
                                         'credit_limit','date_first_order','e_mail_address','fax','phone',
                                            'Sales_Tax_Code','cust_state','zip','terms_code'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
customer

,cust_key,cust_name,active,address_1,address_2,address_3,city,country,credit_limit,date_first_order,e_mail_address,fax,phone,Sales_Tax_Code,cust_state,zip,terms_code
0,2502,Visa 1,None,None,None,None,San Mateo,USA,25877000,None,Cesar.Ramsey@Visa.com,None,None,None,CA,94404,Net60
1,2503,TD Canada Trust 1,None,None,None,None,Toronto,Canada,4248000,None,Hector.Johnson@TDCanadaTrust.com,None,None,None,ON,M4C4X4,Net60
2,2504,Costco 1,None,None,None,None,Issaquah,USA,28950000,None,Phoenix.Harding@Costco.com,None,None,None,WA,98027,Net30
3,2505,Amex 1,None,None,None,None,New York,USA,12273000,None,Isaiah.Keith@Amex.com,None,None,None,NY,10045,COD
4,2506,Best Buy 1,None,None,None,None,Minneapolis,USA,27851000,None,Delaney.Phillips@BestBuy.com,None,None,None,MN,55423,COD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2926,JP Morgan Chase 16,None,None,None,None,New York,USA,18972000,None,Damian.Mcclain@JPMorganChase.com,None,None,None,NY,10017,Net20
496,2927,Neiman Marcus 16,None,None,None,None,Dallas,USA,32615000,None,Sanai.Wang@NeimanMarcus.com,None,None,None,TX,75270,Net20
497,2928,Wells Fargo 11,None,None,None,None,San Francisco,USA,33941000,None,Cecilia.Jennings@WellsFargo.com,None,None,None,CA,94110,Net60
498,2929,Neiman Marcus 17,None,None,None,None,Dallas,USA,3098000,None,Dayami.Drake@NeimanMarcus.com,None,None,None,TX,75270,Net20


In [41]:
# customer transform
#Lọc lấy các cột cần thiết
customer.drop(columns = ['active','address_1','address_2','address_3','date_first_order','fax','phone','Sales_Tax_Code'],inplace = True)
#Kiểm tra giá trị null
customer.info()
customer
#Ta thấy không có giá trị null nào và đang đúng định dạng mong muốn
#Kết thúc Customer Transform

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cust_key        500 non-null    int64 
 1   cust_name       500 non-null    object
 2   city            500 non-null    object
 3   country         500 non-null    object
 4   credit_limit    500 non-null    int64 
 5   e_mail_address  500 non-null    object
 6   cust_state      500 non-null    object
 7   zip             500 non-null    object
 8   terms_code      500 non-null    object
dtypes: int64(2), object(7)
memory usage: 35.3+ KB


,cust_key,cust_name,city,country,credit_limit,e_mail_address,cust_state,zip,terms_code
0,2502,Visa 1,San Mateo,USA,25877000,Cesar.Ramsey@Visa.com,CA,94404,Net60
1,2503,TD Canada Trust 1,Toronto,Canada,4248000,Hector.Johnson@TDCanadaTrust.com,ON,M4C4X4,Net60
2,2504,Costco 1,Issaquah,USA,28950000,Phoenix.Harding@Costco.com,WA,98027,Net30
3,2505,Amex 1,New York,USA,12273000,Isaiah.Keith@Amex.com,NY,10045,COD
4,2506,Best Buy 1,Minneapolis,USA,27851000,Delaney.Phillips@BestBuy.com,MN,55423,COD
...,...,...,...,...,...,...,...,...,...
495,2926,JP Morgan Chase 16,New York,USA,18972000,Damian.Mcclain@JPMorganChase.com,NY,10017,Net20
496,2927,Neiman Marcus 16,Dallas,USA,32615000,Sanai.Wang@NeimanMarcus.com,TX,75270,Net20
497,2928,Wells Fargo 11,San Francisco,USA,33941000,Cecilia.Jennings@WellsFargo.com,CA,94110,Net60
498,2929,Neiman Marcus 17,Dallas,USA,3098000,Dayami.Drake@NeimanMarcus.com,TX,75270,Net20


In [42]:
# customer  Load
#Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_customer_d
    for x in customer.values:
        cur.execute('''INSERT INTO public.w_customer_d(cust_key, cust_name,
        city, country, credit_limit, e_mail_address, cust_state, zip,
        terms_code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Customer location

In [43]:
# Customer location Extract
#Kết nối vào CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện truy vấn lấy tất cả giá trị trong bảng Customer Location
    cur.execute('''SELECT *
                    FROM public."CustLocation";''')
    res = cur.fetchall()
    cust_location=pd.DataFrame(res,columns = ['cust_loc_key','cust_name','active','address_1','address_2','address_3',
                                              'city','country','credit_limit','date_first_order','e_mail_address','fax','phone','sale_tax_code',
                                            'cust_location_state','terms_code','zip','cust_key'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
cust_location

,cust_loc_key,cust_name,active,address_1,address_2,address_3,city,country,credit_limit,date_first_order,e_mail_address,fax,phone,sale_tax_code,cust_location_state,terms_code,zip,cust_key
0,12695,Neiman Marcus 1 - Location 1,None,None,None,None,Canyon Country,USA,10540000,None,Charlie.Baker@NeimanMarcus.com,None,None,None,CA,None,91387,2501
1,12696,Neiman Marcus 1 - Location 2,None,None,None,None,Summerfield,USA,1414000,None,Lilliana.Logan@NeimanMarcus.com,None,None,None,FL,None,34491,2501
2,12697,Visa 1 - Location 1,None,None,None,None,Des Plaines,USA,10342000,None,Ann.Schroeder@Visa.com,None,None,None,IL,None,60016,2502
3,12698,Visa 1 - Location 2,None,None,None,None,Clayton,USA,5285000,None,Cierra.Stephenson@Visa.com,None,None,None,NC,None,27520,2502
4,12753,Wells Fargo 1 - Location 1,None,None,None,None,Hendersonville,USA,11046000,None,Evan.Guerrero@WellsFargo.com,None,None,None,NC,None,28792,2524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,13773,Amex 16 - Location 1,None,None,None,None,Lancaster,USA,8343000,None,Jovanni.Valencia@Amex.com,None,None,None,NY,None,14086,2863
1477,13774,CVS 22 - Location 1,None,None,None,None,Des Plaines,USA,533000,None,Steve.Daugherty@CVS.com,None,None,None,IL,None,60016,2864
1478,13775,CVS 22 - Location 2,None,None,None,None,Muscatine,USA,4996000,None,Hudson.Bryant@CVS.com,None,None,None,IA,None,52761,2864
1479,13776,CVS 22 - Location 3,None,None,None,None,Port Charlotte,USA,10851000,None,Ayaan.Day@CVS.com,None,None,None,FL,None,33952,2864


In [44]:
# customer locationtransform
#Lọc lấy các cột cần thiết
cust_location.drop(columns = ['active','address_1','address_2','address_3','date_first_order','fax','phone','sale_tax_code','terms_code'],inplace = True)
#Kiểm tra giá trị null
cust_location.info()
cust_location
#Ta thấy không có giá trị null nào và đang đúng định dạng mong muốn
#Kết thúc Customer Location Transform

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cust_loc_key         1481 non-null   int64 
 1   cust_name            1481 non-null   object
 2   city                 1481 non-null   object
 3   country              1481 non-null   object
 4   credit_limit         1481 non-null   int64 
 5   e_mail_address       1481 non-null   object
 6   cust_location_state  1481 non-null   object
 7   zip                  1481 non-null   object
 8   cust_key             1481 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 104.3+ KB


,cust_loc_key,cust_name,city,country,credit_limit,e_mail_address,cust_location_state,zip,cust_key
0,12695,Neiman Marcus 1 - Location 1,Canyon Country,USA,10540000,Charlie.Baker@NeimanMarcus.com,CA,91387,2501
1,12696,Neiman Marcus 1 - Location 2,Summerfield,USA,1414000,Lilliana.Logan@NeimanMarcus.com,FL,34491,2501
2,12697,Visa 1 - Location 1,Des Plaines,USA,10342000,Ann.Schroeder@Visa.com,IL,60016,2502
3,12698,Visa 1 - Location 2,Clayton,USA,5285000,Cierra.Stephenson@Visa.com,NC,27520,2502
4,12753,Wells Fargo 1 - Location 1,Hendersonville,USA,11046000,Evan.Guerrero@WellsFargo.com,NC,28792,2524
...,...,...,...,...,...,...,...,...,...
1476,13773,Amex 16 - Location 1,Lancaster,USA,8343000,Jovanni.Valencia@Amex.com,NY,14086,2863
1477,13774,CVS 22 - Location 1,Des Plaines,USA,533000,Steve.Daugherty@CVS.com,IL,60016,2864
1478,13775,CVS 22 - Location 2,Muscatine,USA,4996000,Hudson.Bryant@CVS.com,IA,52761,2864
1479,13776,CVS 22 - Location 3,Port Charlotte,USA,10851000,Ayaan.Day@CVS.com,FL,33952,2864


In [45]:
# customer location Load
#Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_cust_location_d
    for x in cust_location.values:
        cur.execute('''INSERT INTO public.w_cust_location_d(cust_loc_key, cust_name, city,
        country, credit_limit, e_mail_address, cust_location_state,
        zip, cust_key)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Machine Type

In [46]:
# machine type extract
#Kết nối CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện ytuy vấn lấy tất cả giá trị trong bảng machine type
    cur.execute('''SELECT "Machine_Type_Id", "Manufacturer", "Model",
    "Rate_Per_Hour", "Number_Of_Machines"
	FROM public."MachineType";''')
    res = cur.fetchall()
    #Chuyển thành dataframe
    machine=pd.DataFrame(res,columns = ['machine_type_id','manufacturer','machine_model','rate_per_hour',
                                         'number_of_machines'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

machine

,machine_type_id,manufacturer,machine_model,rate_per_hour,number_of_machines
0,1,Manufacturer A,Model 1,40,5
1,2,Manufacturer A,Model 2,20,3
2,3,Manufacturer B,Model 101,100,12
3,4,Manufacturer C,Model 201,50,4
4,5,Manufacturer C,Model 202,30,2
5,6,Manufacturer C,Model 203,8,1
6,7,Manufacturer D,Model 301,60,4


In [47]:
# machine Transform
#Ta thấy dữ liệu có 6 dòng, đúng định dạng, đúng thiết kế các cột cần thiết
# Kết thúc machine Transform

In [48]:
# machine load 
##Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_machine_type_d
    for x in machine.values:
        cur.execute('''INSERT INTO public.w_machine_type_d(
	machine_type_id, manufacturer, machine_model, rate_per_hour,
    number_of_machines)
	    VALUES (%s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Job _Phần bảng Fact

In [52]:
# job extract 
#Kết nối CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện lấy các cột cần thiết trong bảng Job, 
    # vì bảng này có nhiều dữ liệu nên để tối ưu hóa thời gian extract ta chỉ lấy các cột cần thiết
    cur.execute('''SELECT "Job_Id", "Contract_Date", "Sales_Agent_Id",
                  "Sales_Class_Id","Location_Id","Cust_Key",
                  "Date_Promised","Date_Ship_By", "Number_Of_Subjobs", 
                  "Unit_Price","Quantity_Ordered", "Quotation_Ordered" 
                  FROM public."Job";''')
    res = cur.fetchall()
    #Chuyển thành dataframe
    job=pd.DataFrame(res,columns = ['job_id','contract_date','Sales_Agent_Id',
                                    'Sales_Class_Id','Location_Id',
                                    'Cust_Key','Date_Promised','Date_Ship_By',
                                    'Number_Of_Subjobs','Unit_Price',
                                   'Quantity_Ordered','Quotation_Ordered'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
job

,job_id,contract_date,Sales_Agent_Id,Sales_Class_Id,Location_Id,Cust_Key,Date_Promised,Date_Ship_By,Number_Of_Subjobs,Unit_Price,Quantity_Ordered,Quotation_Ordered
0,342040,2013-03-04,1462,4,12,2673,2013-04-09,2013-04-01,3,1,933400,933400
1,342041,2013-05-02,1446,2,11,2761,2013-05-22,2013-05-15,7,1,1256600,1208300
2,342042,2015-02-03,1286,1,12,2800,2015-02-27,2015-02-19,4,2,418300,366900
3,342043,2013-05-09,1502,5,2,2507,2013-05-30,2013-05-24,1,1,198500,189000
4,342044,2013-12-11,1616,5,10,2676,2014-01-10,2014-01-03,2,1,362900,442500
...,...,...,...,...,...,...,...,...,...,...,...,...
2564,340628,2015-01-01,1581,6,9,2554,2015-01-28,2015-01-21,4,1,377400,373700
2565,340629,2014-04-03,1471,3,8,2747,2014-04-30,2014-04-25,1,1,220600,245100
2566,340630,2014-02-25,1195,1,10,2671,2014-03-27,2014-03-18,5,2,754900,645200
2567,340631,2014-09-23,1327,3,6,2501,2014-10-30,2014-10-22,1,1,366500,333200


In [53]:
#job Transformation
#Kiểm tra info của job
job.info()
#Ta thấy không có cột dữ liệu nào bị null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2569 entries, 0 to 2568
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_id             2569 non-null   int64 
 1   contract_date      2569 non-null   object
 2   Sales_Agent_Id     2569 non-null   int64 
 3   Sales_Class_Id     2569 non-null   int64 
 4   Location_Id        2569 non-null   int64 
 5   Cust_Key           2569 non-null   int64 
 6   Date_Promised      2569 non-null   object
 7   Date_Ship_By       2569 non-null   object
 8   Number_Of_Subjobs  2569 non-null   int64 
 9   Unit_Price         2569 non-null   int64 
 10  Quantity_Ordered   2569 non-null   int64 
 11  Quotation_Ordered  2569 non-null   int64 
dtypes: int64(9), object(3)
memory usage: 241.0+ KB


In [54]:

#Do các cột chứa giá trị về thời gian đang chưa ở định dạng mong muốn nên sẽ chuyển thành chuỗi
#Chuyển đổi cột Contract_date
job['contract_date']=job['contract_date'].astype(str)
#Chuyển về định dạng YYYYMMDD (Lấy 4 ký tự đầu (năm), 2 ký tự từ vị trí 5 (tháng), ký tự từ vị trí 8 đến hết (ngày))
job['contract_date']=job['contract_date'].str.slice(0,4) + job['contract_date'].str.slice(5,7) + job['contract_date'].str.slice(8)

#Chuyển đổi cột Date_Promised
job['Date_Promised']=job['Date_Promised'].astype(str)
job['Date_Promised']=job['Date_Promised'].str.slice(0,4) + job['Date_Promised'].str.slice(5,7) + job['Date_Promised'].str.slice(8)

#Chuyển đổi cột Date_ship_by
job['Date_Ship_By']=job['Date_Ship_By'].astype(str)
job['Date_Ship_By']=job['Date_Ship_By'].str.slice(0,4) + job['Date_Ship_By'].str.slice(5,7) + job['Date_Ship_By'].str.slice(8)

job['contract_date']=job['contract_date']
job['Date_Promised']=job['Date_Promised']
job['Date_Ship_By']=job['Date_Ship_By']
#Kiểm tra lại info Job
job.info()
job.head()
#Các cột thời gian đã đúng định dạng

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2569 entries, 0 to 2568
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_id             2569 non-null   int64 
 1   contract_date      2569 non-null   object
 2   Sales_Agent_Id     2569 non-null   int64 
 3   Sales_Class_Id     2569 non-null   int64 
 4   Location_Id        2569 non-null   int64 
 5   Cust_Key           2569 non-null   int64 
 6   Date_Promised      2569 non-null   object
 7   Date_Ship_By       2569 non-null   object
 8   Number_Of_Subjobs  2569 non-null   int64 
 9   Unit_Price         2569 non-null   int64 
 10  Quantity_Ordered   2569 non-null   int64 
 11  Quotation_Ordered  2569 non-null   int64 
dtypes: int64(9), object(3)
memory usage: 241.0+ KB


,job_id,contract_date,Sales_Agent_Id,Sales_Class_Id,Location_Id,Cust_Key,Date_Promised,Date_Ship_By,Number_Of_Subjobs,Unit_Price,Quantity_Ordered,Quotation_Ordered
0,342040,20130304,1462,4,12,2673,20130409,20130401,3,1,933400,933400
1,342041,20130502,1446,2,11,2761,20130522,20130515,7,1,1256600,1208300
2,342042,20150203,1286,1,12,2800,20150227,20150219,4,2,418300,366900
3,342043,20130509,1502,5,2,2507,20130530,20130524,1,1,198500,189000
4,342044,20131211,1616,5,10,2676,20140110,20140103,2,1,362900,442500


In [55]:
# job load 
##Kết nối vào CSDL lưu trữ Data Warehouse
#Kết nối CSDL erp dw 
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_Job_F
    for x in job.values:
        cur.execute('''INSERT INTO public.w_job_f(job_id, contract_date, sales_agent_id, sales_class_id,
        location_id, cust_id_ordered_by,date_promised, date_ship_by, number_of_subjobs, unit_price,
        quantity_ordered, quote_qty)
    VALUES (%s, %s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],
                                            x[8],x[9],x[10],x[11]))
        conn.commit()
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Sub job Fact

In [56]:
# Subjob Extract
#Kết nối CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện lấy các cột cần thiết trong bảng Sub Job, ngoài ra cần nối thêm với bảng Job để lấy thêm cột Cust_key
    # vì bảng này có nhiều dữ liệu nên để tối ưu hóa thời gian extract ta chỉ lấy các cột cần thiết
    cur.execute('''SELECT "Sub_Job_Id", "Cost_Labor", "Cost_Material", "Cost_Overhead",
                    "Machine_Hours", "Quantity_Produced", "Sub_Job_Amount", s."Job_Id",
                    "Machine_Type_Id","Sales_Class_Id",
                    "Date_Prod_Begin", "Date_Prod_End",
                    "Location_Id" ,"Cust_Key"
                    FROM "Subjob" s , "Job" j 
                    where s."Job_Id" = j."Job_Id";''')
    res = cur.fetchall()
      #Chuyển thành dataframe
    sub_job=pd.DataFrame(res,columns = ['sub_job_id', 'cost_labor', 'cost_material', 'cost_overhead',
                                        'machine_hours','Quantity_Produced', 'Sub_Job_Amount','Job_Id',
                                        'Machine_Type_Id','Sales_Class_Id','Date_Prod_Begin','Date_Prod_End',
                                       'Location_Id','Cust_Key'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
sub_job

,sub_job_id,cost_labor,cost_material,cost_overhead,machine_hours,Quantity_Produced,Sub_Job_Amount,Job_Id,Machine_Type_Id,Sales_Class_Id,Date_Prod_Begin,Date_Prod_End,Location_Id,Cust_Key
0,1253661,36706,6313,9470,595.0,206400,160992,340015,2,4,2014-09-24,2014-10-22,12,2668
1,1253662,12284,1842,1280,26.0,57900,45162,340015,3,4,2014-10-03,2014-10-16,12,2668
2,1253663,83875,4610,10262,440.0,205900,312968,340016,7,1,2013-09-04,2013-09-09,4,2732
3,1253664,35052,12015,26743,1712.0,205900,312968,340016,5,1,2013-09-02,2013-09-13,4,2732
4,1253665,38808,17958,10547,7234.0,205900,312968,340016,6,1,2013-09-03,2013-09-13,4,2732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8052,1261635,82845,13339,4934,311.0,394200,291708,342560,4,4,2014-03-13,2014-04-04,10,2994
8053,1261636,31504,12189,32956,2001.0,394200,291708,342560,2,4,2014-03-11,2014-03-28,10,2994
8054,1261637,10589,2854,5300,239.0,79500,58830,342560,2,4,2014-03-11,2014-03-21,10,2994
8055,1261638,74636,53449,65326,1598.0,423300,643416,342561,1,1,2013-06-12,2013-07-15,5,2713


In [57]:
#Subjob Transformation
#Kiểm tra info của sub job
sub_job.info()
#Ta thấy không có cột dữ liệu nào bị null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sub_job_id         8057 non-null   int64  
 1   cost_labor         8057 non-null   int64  
 2   cost_material      8057 non-null   int64  
 3   cost_overhead      8057 non-null   int64  
 4   machine_hours      8057 non-null   float64
 5   Quantity_Produced  8057 non-null   int64  
 6   Sub_Job_Amount     8057 non-null   int64  
 7   Job_Id             8057 non-null   int64  
 8   Machine_Type_Id    8057 non-null   int64  
 9   Sales_Class_Id     8057 non-null   int64  
 10  Date_Prod_Begin    8057 non-null   object 
 11  Date_Prod_End      8057 non-null   object 
 12  Location_Id        8057 non-null   int64  
 13  Cust_Key           8057 non-null   int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 881.4+ KB


In [58]:
# Sub job Transformation 
#Do các cột chứa giá trị về thời gian đang chưa ở định dạng mong muốn nên sẽ chuyển thành chuỗi
#Chuyển đổi cột Date_Prod_Begin
sub_job["Date_Prod_Begin"]=sub_job['Date_Prod_Begin'].astype(str)
sub_job['Date_Prod_Begin']=sub_job['Date_Prod_Begin'].str.slice(0,4) + sub_job['Date_Prod_Begin'].str.slice(5,7)+ sub_job['Date_Prod_Begin'].str.slice(8)

#Chuyển đổi cột Date_Prod_End
sub_job['Date_Prod_End']=sub_job['Date_Prod_End'].astype(str)
sub_job['Date_Prod_End']=sub_job['Date_Prod_End'].str.slice(0,4) + sub_job['Date_Prod_End'].str.slice(5,7) + sub_job['Date_Prod_End'].str.slice(8)

#Chuyển đổi cột Date_Prod_Begin
sub_job['Date_Prod_Begin']=sub_job['Date_Prod_Begin']
sub_job['Date_Prod_End']=sub_job['Date_Prod_End']

#Kiểm tra lại sub job
sub_job.info()
sub_job.head()
#Ta thấy các cột thời gian đã đúng định dạng

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sub_job_id         8057 non-null   int64  
 1   cost_labor         8057 non-null   int64  
 2   cost_material      8057 non-null   int64  
 3   cost_overhead      8057 non-null   int64  
 4   machine_hours      8057 non-null   float64
 5   Quantity_Produced  8057 non-null   int64  
 6   Sub_Job_Amount     8057 non-null   int64  
 7   Job_Id             8057 non-null   int64  
 8   Machine_Type_Id    8057 non-null   int64  
 9   Sales_Class_Id     8057 non-null   int64  
 10  Date_Prod_Begin    8057 non-null   object 
 11  Date_Prod_End      8057 non-null   object 
 12  Location_Id        8057 non-null   int64  
 13  Cust_Key           8057 non-null   int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 881.4+ KB


,sub_job_id,cost_labor,cost_material,cost_overhead,machine_hours,Quantity_Produced,Sub_Job_Amount,Job_Id,Machine_Type_Id,Sales_Class_Id,Date_Prod_Begin,Date_Prod_End,Location_Id,Cust_Key
0,1253661,36706,6313,9470,595.0,206400,160992,340015,2,4,20140924,20141022,12,2668
1,1253662,12284,1842,1280,26.0,57900,45162,340015,3,4,20141003,20141016,12,2668
2,1253663,83875,4610,10262,440.0,205900,312968,340016,7,1,20130904,20130909,4,2732
3,1253664,35052,12015,26743,1712.0,205900,312968,340016,5,1,20130902,20130913,4,2732
4,1253665,38808,17958,10547,7234.0,205900,312968,340016,6,1,20130903,20130913,4,2732


In [59]:
# subjob load
# ##Kết nối vào CSDL lưu trữ Data Warehouse
#Kết nối CSDL erp dw  
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_Sub_Job_F
    for x in sub_job.values:
        cur.execute('''INSERT INTO public.w_sub_job_f(
	sub_job_id, cost_labor, cost_material, cost_overhead, machine_hours,
    quantity_produced, sub_job_amount, job_id, machine_type_id, sales_class_id,
    date_prod_begin, date_prod_end, location_id, cust_id_ordered_by)

	VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s);''', (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],
                    x[8],x[9],x[10],x[11],x[12],x[13]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Invoice

In [61]:
# invoice extract
#Kết nối CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện lấy các cột cần thiết trong bảng Invoice,
    #  ngoài ra cần nối thêm với bảng Job, Sub Job để lấy thêm cột Sales_Class_Id, Sales_Agent_Id,Location_Id khớp với thông tin trên hóa đơn
    # vì bảng này có nhiều dữ liệu nên để tối ưu hóa thời gian extract ta chỉ lấy các cột cần thiết
    cur.execute('''SELECT i."Invoice_Id","Invoice_Quantity","Invoice_Amount",
                "Invoice_Shipped","Sales_Class_Id",
                "Date_Invoiced", "Date_Due", j."Cust_Key",
                "Sales_Agent_Id", "Location_Id"
                FROM "Invoice" i,shipment sh,"Subjob" s,"Job" j
                where i."Invoice_Id"=sh."Invoice_Id" and 
                sh."Sub_Job_Id" = s."Sub_Job_Id" and 
                s."Job_Id" = j."Job_Id";''')
    res = cur.fetchall()
    #Chuyển thành dataframe
    invoice=pd.DataFrame(res,columns = ['Invoice_Id','Invoice_Quantity','Invoice_Amount',
                                        'Invoice_Shipped',
                                         'Sales_Class_Id','Date_Invoiced',
                                            'Date_Due','Cust_Key','Sales_Agent_Id','Location_Id'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
invoice

,Invoice_Id,Invoice_Quantity,Invoice_Amount,Invoice_Shipped,Sales_Class_Id,Date_Invoiced,Date_Due,Cust_Key,Sales_Agent_Id,Location_Id
0,0,NaN,NaN,NaN,4,None,None,2592,1451,6
1,4015557,129500.0,97125.0,129500.0,4,2015-02-17,2015-05-12,2592,1451,6
2,4015558,72100.0,54075.0,72100.0,4,2015-03-17,2015-06-09,2592,1451,6
3,0,NaN,NaN,NaN,4,None,None,2592,1451,6
4,4015559,72100.0,54075.0,72100.0,4,2015-02-17,2015-05-12,2592,1451,6
...,...,...,...,...,...,...,...,...,...,...
24230,4036096,22790.0,19610.0,26500.0,4,2014-04-23,2014-06-04,2994,1246,10
24231,4036097,423300.0,643416.0,423300.0,1,2013-07-16,2013-07-17,2713,1331,5
24232,4036098,118000.0,179360.0,118000.0,1,2013-07-16,2013-07-17,2713,1331,5
24233,4036099,118000.0,179360.0,118000.0,1,2013-07-16,2013-07-17,2713,1331,5


In [62]:
# invoice Transforamtion
#Do các cột chứa giá trị về thời gian đang chưa ở định dạng mong muốn nên sẽ chuyển thành chuỗi
#Chuyển đổi cột Date_Invoice
invoice['Date_Invoiced']=invoice['Date_Invoiced'].astype(str)
invoice['Date_Invoiced']=invoice['Date_Invoiced'].str.slice(0,4) + invoice['Date_Invoiced'].str.slice(5,7)+ invoice['Date_Invoiced'].str.slice(8)

#Chuyển đổi cột Date_Due
invoice['Date_Due']= invoice['Date_Due'].astype(str)
invoice['Date_Due']=invoice['Date_Due'].str.slice(0,4) + invoice['Date_Due'].str.slice(5,7)+ invoice['Date_Due'].str.slice(8)
invoice

,Invoice_Id,Invoice_Quantity,Invoice_Amount,Invoice_Shipped,Sales_Class_Id,Date_Invoiced,Date_Due,Cust_Key,Sales_Agent_Id,Location_Id
0,0,NaN,NaN,NaN,4,None,None,2592,1451,6
1,4015557,129500.0,97125.0,129500.0,4,20150217,20150512,2592,1451,6
2,4015558,72100.0,54075.0,72100.0,4,20150317,20150609,2592,1451,6
3,0,NaN,NaN,NaN,4,None,None,2592,1451,6
4,4015559,72100.0,54075.0,72100.0,4,20150217,20150512,2592,1451,6
...,...,...,...,...,...,...,...,...,...,...
24230,4036096,22790.0,19610.0,26500.0,4,20140423,20140604,2994,1246,10
24231,4036097,423300.0,643416.0,423300.0,1,20130716,20130717,2713,1331,5
24232,4036098,118000.0,179360.0,118000.0,1,20130716,20130717,2713,1331,5
24233,4036099,118000.0,179360.0,118000.0,1,20130716,20130717,2713,1331,5


In [63]:
#Invoice Transformation
#Ta thấy có xuất hiện giá trị Invoice_Id = 0 -> xóa các invoice_id có giá trị = 0
invoice= invoice.loc[invoice['Invoice_Id'] != 0]
invoice

,Invoice_Id,Invoice_Quantity,Invoice_Amount,Invoice_Shipped,Sales_Class_Id,Date_Invoiced,Date_Due,Cust_Key,Sales_Agent_Id,Location_Id
1,4015557,129500.0,97125.0,129500.0,4,20150217,20150512,2592,1451,6
2,4015558,72100.0,54075.0,72100.0,4,20150317,20150609,2592,1451,6
4,4015559,72100.0,54075.0,72100.0,4,20150217,20150512,2592,1451,6
5,4015560,72100.0,54075.0,72100.0,4,20150317,20150609,2592,1451,6
6,4015561,72100.0,54075.0,72100.0,4,20150317,20150609,2592,1451,6
...,...,...,...,...,...,...,...,...,...,...
24230,4036096,22790.0,19610.0,26500.0,4,20140423,20140604,2994,1246,10
24231,4036097,423300.0,643416.0,423300.0,1,20130716,20130717,2713,1331,5
24232,4036098,118000.0,179360.0,118000.0,1,20130716,20130717,2713,1331,5
24233,4036099,118000.0,179360.0,118000.0,1,20130716,20130717,2713,1331,5


In [65]:
# invoice Load 
 ##Kết nối vào CSDL lưu trữ Data Warehouse
#Kết nối CSDL erp dw 
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
     # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_Invoice_F
    for x in invoice.values:
        cur.execute('''INSERT INTO public.w_invoiceline_f(invoice_id, invoice_quantity,
        invoice_amount, quantity_shipped, sales_class_id, invoice_sent_date,
        invoice_due_date, cust_key, sales_agent_id, location_id)
	       VALUES (%s, %s, %s, %s, %s, %s, %s,%s,%s,%s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

duplicate key value violates unique constraint "w_invoiceline_f_pk"
DETAIL:  Key (invoice_id)=(4015557) already exists.



In [66]:
# shipment extract
#Kết nối CSDL erp
hostname = 'localhost'
database = 'erp'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    #Thực hiện lấy các cột cần thiết trong bảng Shipment,
    #  ngoài ra cần nối thêm với bảng Job, Sub Job để lấy thêm cột Sales_Class_Id,Location_Id khớp với thông tin giao hàng
    # vì bảng này có nhiều dữ liệu nên để tối ưu hóa thời gian extract ta chỉ lấy các cột cần thiết
    cur.execute('''SELECT "Shipment_Id","Actual_Quantity","Requested_Quantity",
                  "Boexs","Quantity_Per_Box","Quantity_Per_Partial_Box",
                  "Shipment_Amount",s."Sub_Job_Id" , j."Sales_Class_Id","Location_Id",
                  "Cust_Loc_Key","Actual_Ship_Date", "Requested_Ship_Date",
                  "Invoice_Id" 
                    FROM shipment sh ,"Subjob" s,"Job" j
                    where sh."Sub_Job_Id"=s."Sub_Job_Id"
                    and s."Job_Id" = j."Job_Id"''')
    res = cur.fetchall()
    #chuyển thành data frame
    shipment=pd.DataFrame(res,columns = ['Shipment_Id','Actual_Quantity','Requested_Quantity',
                                         'Boexs','Quantity_Per_Box','Quantity_Per_Partial_Box',
                                            'Shipment_Amount','Sub_Job_Id',
                                         'Sales_Class_Id','Location_Id',
                                   'Cust_Loc_Key','Actual_Ship_Date','Requested_Ship_Date','Invoice_Id'])
    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()
shipment

,Shipment_Id,Actual_Quantity,Requested_Quantity,Boexs,Quantity_Per_Box,Quantity_Per_Partial_Box,Shipment_Amount,Sub_Job_Id,Sales_Class_Id,Location_Id,Cust_Loc_Key,Actual_Ship_Date,Requested_Ship_Date,Invoice_Id
0,5275763,129500,129500,43,3000,500,97125,1253614,4,6,12948,2015-01-29,2015-01-26,0
1,5275764,129500,129500,49,2600,2100,97125,1253614,4,6,12948,2015-01-26,2015-01-26,4015557
2,5275765,72100,72100,28,2500,2100,54075,1253615,4,6,12948,2015-01-29,2015-01-29,4015558
3,5275766,72100,72100,14,4900,3500,54075,1253615,4,6,12948,2015-01-28,2015-01-28,0
4,5275767,72100,69937,18,4000,100,54075,1253615,4,6,12948,2015-01-30,2015-01-29,4015559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24230,5299896,26500,27295,5,4800,2500,19610,1261637,4,10,14150,2014-03-13,2014-03-13,4036096
24231,5299897,423300,423300,384,1100,900,643416,1261638,1,5,13296,2013-06-14,2013-06-14,4036097
24232,5299898,118000,118000,45,2600,1000,179360,1261639,1,5,13296,2013-06-14,2013-06-14,4036098
24233,5299899,118000,100300,30,3900,1000,179360,1261639,1,5,13296,2013-06-18,2013-06-13,4036099


In [67]:
#Subjob Transformation
#Kiểm tra info của sub job
shipment.info()
#Ta thấy không có cột dữ liệu nào bị null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24235 entries, 0 to 24234
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Shipment_Id               24235 non-null  int64 
 1   Actual_Quantity           24235 non-null  int64 
 2   Requested_Quantity        24235 non-null  int64 
 3   Boexs                     24235 non-null  int64 
 4   Quantity_Per_Box          24235 non-null  int64 
 5   Quantity_Per_Partial_Box  24235 non-null  int64 
 6   Shipment_Amount           24235 non-null  int64 
 7   Sub_Job_Id                24235 non-null  int64 
 8   Sales_Class_Id            24235 non-null  int64 
 9   Location_Id               24235 non-null  int64 
 10  Cust_Loc_Key              24235 non-null  int64 
 11  Actual_Ship_Date          24235 non-null  object
 12  Requested_Ship_Date       24235 non-null  object
 13  Invoice_Id                24235 non-null  int64 
dtypes: int64(12), object(2

In [68]:
# invoice Transforamtion
#Do các cột chứa giá trị về thời gian đang chưa ở định dạng mong muốn nên sẽ chuyển thành chuỗi
#Chuyển đổi cột Actual_Ship_Date
shipment['Actual_Ship_Date']=shipment['Actual_Ship_Date'].astype(str)
shipment['Actual_Ship_Date']=shipment['Actual_Ship_Date'].str.slice(0,4) + shipment['Actual_Ship_Date'].str.slice(5,7)+ shipment['Actual_Ship_Date'].str.slice(8)

#Chuyển đổi cột Requested_Ship_Date
shipment['Requested_Ship_Date']=shipment['Requested_Ship_Date'].astype(str)
shipment['Requested_Ship_Date']=shipment['Requested_Ship_Date'].str.slice(0,4) + shipment['Requested_Ship_Date'].str.slice(5,7)+ shipment['Requested_Ship_Date'].str.slice(8)

shipment

,Shipment_Id,Actual_Quantity,Requested_Quantity,Boexs,Quantity_Per_Box,Quantity_Per_Partial_Box,Shipment_Amount,Sub_Job_Id,Sales_Class_Id,Location_Id,Cust_Loc_Key,Actual_Ship_Date,Requested_Ship_Date,Invoice_Id
0,5275763,129500,129500,43,3000,500,97125,1253614,4,6,12948,20150129,20150126,0
1,5275764,129500,129500,49,2600,2100,97125,1253614,4,6,12948,20150126,20150126,4015557
2,5275765,72100,72100,28,2500,2100,54075,1253615,4,6,12948,20150129,20150129,4015558
3,5275766,72100,72100,14,4900,3500,54075,1253615,4,6,12948,20150128,20150128,0
4,5275767,72100,69937,18,4000,100,54075,1253615,4,6,12948,20150130,20150129,4015559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24230,5299896,26500,27295,5,4800,2500,19610,1261637,4,10,14150,20140313,20140313,4036096
24231,5299897,423300,423300,384,1100,900,643416,1261638,1,5,13296,20130614,20130614,4036097
24232,5299898,118000,118000,45,2600,1000,179360,1261639,1,5,13296,20130614,20130614,4036098
24233,5299899,118000,100300,30,3900,1000,179360,1261639,1,5,13296,20130618,20130613,4036099


In [69]:
# shipment load 
 ##Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_shipment_F
    for x in shipment.values:
        cur.execute('''INSERT INTO public.w_job_shipment_f(job_shipment_id,
        actual_quantity, requested_quantity, boxes, quantity_per_box,
        quantity_per_partial_box, shipped_amount, sub_job_id, 
        sales_class_id, location_id, cust_id_ship_to, actual_ship_date,
        requested_ship_date, invoice_id)
	VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],
                                            x[8],x[9],x[10],x[11],x[12],x[13]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Lead

In [70]:
# Lead Extract
#Đọc file lead 
lead = pd.read_csv('lead.csv')
#Kiểm tra lead
lead

,lead_id,quote_qty,quote_price,lead_success,job_id,created_date,cust_id,location_id,sales_agent_id,sales_class_id
0,424767,508600,1.54,Y,341546.0,20131113,2887,3,1251,2
1,424768,780100,1.50,Y,341547.0,20140205,2793,10,1150,1
2,424769,84000,0.80,N,NaN,20131025,2895,5,1426,3
3,424770,152600,0.79,Y,341548.0,20141008,2824,11,1606,5
4,424771,159500,0.79,Y,341549.0,20140312,2763,6,1497,5
...,...,...,...,...,...,...,...,...,...,...
2995,424102,243500,1.58,Y,340982.0,20140828,2839,6,1192,2
2996,424103,158400,0.79,Y,340983.0,20140530,2666,2,1572,3
2997,424104,1081700,0.74,Y,340984.0,20131204,2973,1,1505,5
2998,424105,134900,0.79,Y,340985.0,20130319,2899,9,1240,4


In [71]:
#Lead Transform
#Kiểm tra file
lead.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   lead_id         3000 non-null   int64  
 1   quote_qty       3000 non-null   int64  
 2   quote_price     3000 non-null   float64
 3   lead_success    3000 non-null   object 
 4   job_id          2569 non-null   float64
 5   created_date    3000 non-null   int64  
 6   cust_id         3000 non-null   int64  
 7   location_id     3000 non-null   int64  
 8   sales_agent_id  3000 non-null   int64  
 9   sales_class_id  3000 non-null   int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 234.5+ KB


In [79]:
import numpy as np

In [72]:
#Điển các giá trị null của trường job_id = 0
lead['job_id'] = lead['job_id'].fillna(0)
#Chuyển trường job_id sang kiểu int 
lead['job_id']=lead['job_id'].astype(int)
lead

,lead_id,quote_qty,quote_price,lead_success,job_id,created_date,cust_id,location_id,sales_agent_id,sales_class_id
0,424767,508600,1.54,Y,341546,20131113,2887,3,1251,2
1,424768,780100,1.50,Y,341547,20140205,2793,10,1150,1
2,424769,84000,0.80,N,0,20131025,2895,5,1426,3
3,424770,152600,0.79,Y,341548,20141008,2824,11,1606,5
4,424771,159500,0.79,Y,341549,20140312,2763,6,1497,5
...,...,...,...,...,...,...,...,...,...,...
2995,424102,243500,1.58,Y,340982,20140828,2839,6,1192,2
2996,424103,158400,0.79,Y,340983,20140530,2666,2,1572,3
2997,424104,1081700,0.74,Y,340984,20131204,2973,1,1505,5
2998,424105,134900,0.79,Y,340985,20130319,2899,9,1240,4


In [73]:
# Lead Load 
 ##Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim W_lead_F
    for x in lead.values:
        cur.execute('''INSERT INTO public.w_lead_f(lead_id, quote_qty, 
        quote_price, lead_success, job_id,
    created_date, cust_id, location_id, sales_agent_id, sales_class_id)
	VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],
                                            x[8],x[9]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

Cost

In [92]:
# Cost Extract
#Đọc dữ liệu từ file cost_summary
cost = pd.read_csv('cost_summary.csv')
cost

,financial_summary_cost_id,actual_units,actual_labor_cost,actual_material_cost,actual_machine_cost,actual_overhead_cost,budget_units,budget_labor_cost,budget_material_cost,budget_machine_cost,budget_overhead_cost,location_id,machine_type_id,sales_class_id,report_begin_date_id,report_end_date_id
0,201402020806,374034,83466,8746.18,80194,17383.53,1856866,539528,107898.83,260941,129619.58,8,6,2,20140203,20140228
1,201401020806,0,0,0.00,0,0.00,1856866,539528,107898.83,260941,129619.58,8,6,2,20140101,20140131
2,201412020806,1780313,519576,102878.37,251971,124538.41,1856866,539528,107898.83,260941,129619.58,8,6,2,20141201,20141231
3,201412020807,1919451,630680,116158.32,239072,159629.97,2001987,657105,120235.48,250739,165488.39,8,7,2,20141201,20141231
4,201408020901,843638,202791,48348.83,204562,42269.71,892737,215040,51607.54,216304,44429.69,9,1,2,20140801,20140829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,201307060904,674042,83104,19689.84,70533,30768.39,888637,100315,29245.25,87279,38380.31,9,4,6,20130701,20130731
13436,201307060906,1276159,262922,44286.34,78591,52423.86,1472837,292611,50132.11,87797,57832.79,9,6,6,20130701,20130731
13437,201307060907,453654,66642,20861.40,34992,12203.07,709959,102952,27308.47,60015,25079.64,9,7,6,20130701,20130731
13438,201307061001,586966,100734,19759.56,52548,17559.33,893709,149786,26250.91,80812,27564.81,10,1,6,20130701,20130731


In [99]:
#Cost Transformation
#Kiểm tra info của cost
cost['financial_summary_cost_id']=cost['financial_summary_cost_id'].astype(str)
cost.info()
#Ta thấy không có cột dữ liệu nào bị null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13440 entries, 0 to 13439
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   financial_summary_cost_id  13440 non-null  object 
 1   actual_units               13440 non-null  int64  
 2   actual_labor_cost          13440 non-null  int64  
 3   actual_material_cost       13440 non-null  float64
 4   actual_machine_cost        13440 non-null  int64  
 5   actual_overhead_cost       13440 non-null  float64
 6   budget_units               13440 non-null  int64  
 7   budget_labor_cost          13440 non-null  int64  
 8   budget_material_cost       13440 non-null  float64
 9   budget_machine_cost        13440 non-null  int64  
 10  budget_overhead_cost       13440 non-null  float64
 11  location_id                13440 non-null  int64  
 12  machine_type_id            13440 non-null  int64  
 13  sales_class_id             13440 non-null  int

In [101]:
# Cost Load 
 ##Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
    # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim w_financial_summary_cost_f
    for x in cost.values:
        cur.execute('''INSERT INTO public.w_financial_summary_cost_f(
	financial_summary_cost_id, actual_units, actual_labor_cost,
    actual_material_cost, actual_machine_cost, actual_overhead_cost,
    budget_units, budget_labor_cost, budget_material_cost, budget_machine_cost,
    budget_overhead_cost, location_id, machine_type_id, sales_class_id,
    report_begin_date_id, report_end_date_id)
	VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],
                                            x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()

In [102]:
# Sales Extract
#Đọc dữ liệu từ file sales summary
sales = pd.read_csv('sales summary.csv')
sales

,financial_summary_sales_id,actual_units,actual_amount,forcast_unit,forcast_amount,location_id,sales_class_id,report_begin_date_id,report_end_date_id
0,2014120112,5695716,8905778.44,5340872,8350948.0,12,1,20141201,20141231
1,2014120201,6113446,9551616.09,6661822,10408396.0,1,2,20141201,20141231
2,2014120202,6305461,9774233.50,6207095,9621755.0,2,2,20141201,20141231
3,2014120203,11098823,17013058.95,10857978,16643876.0,3,2,20141201,20141231
4,2014120204,5745211,8997036.30,5625136,8808998.0,4,2,20141201,20141231
...,...,...,...,...,...,...,...,...,...
2155,2013050605,812479,631642.32,3265116,2522140.0,5,6,20130501,20130531
2156,2013120605,3486510,2693154.87,3265116,2522140.0,5,6,20131202,20131231
2157,2013020606,0,0.00,7328728,5821472.0,6,6,20130201,20130228
2158,2013120606,7012466,5570253.86,7328728,5821472.0,6,6,20131202,20131231


In [105]:
#Sale Transformation
sales['financial_summary_sales_id']=sales['financial_summary_sales_id'].astype(str)
#Kiểm tra info của sale
sales.info()
#Ta thấy không có cột dữ liệu nào bị null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   financial_summary_sales_id  2160 non-null   object 
 1   actual_units                2160 non-null   int64  
 2   actual_amount               2160 non-null   float64
 3   forcast_unit                2160 non-null   int64  
 4   forcast_amount              2160 non-null   float64
 5   location_id                 2160 non-null   int64  
 6   sales_class_id              2160 non-null   int64  
 7   report_begin_date_id        2160 non-null   int64  
 8   report_end_date_id          2160 non-null   int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 152.0+ KB


In [106]:
# sales Load 
 ##Kết nối vào CSDL lưu trữ Data Warehouse
hostname = 'localhost'
database = 'erp dw'
username = 'postgres'
pwd = '123'
port_id = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
    cur = conn.cursor()
     # Thực hiện vòng for insert các giá trị trong dataframe vào bảng dim w_financial_summary_sales_f
    for x in sales.values:
        cur.execute('''INSERT INTO public.w_financial_summary_sales_f(
	financial_summary_sales_id, actual_units, actual_amount,
    forcast_unit, forcast_amount, location_id, sales_class_id,
    report_begin_date_id, report_end_date_id)
	VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);''',(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]))
        conn.commit();
   

    conn.close()
except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:    
        conn.close()